# Chatbot con un LLM - Interfaces de conversación 

> *Intenta ejecutar este notebook en un Colab o en tu maquina con JupyterLab*

En este cuaderno, vamos a crear un chatbot utilizando algunos de los modelos fundacionales de Amazon Bedrock

## Introducción

Los bot conversacionales (chatbot) surgen en los años 60 y simulan la conversación que se lleva a cabo con otra persona. Se trata de programas que utilizan machine learning y procesamiento de lenguaje natural para interpretar las preguntas de los usuarios y responder de forma automatica. 
Actualmente los chatbot son utilizados en diferentes aplicaciones y dominios de negocio como el servicio al cliente o las ventas para proveer respuestas eficientes a los clientes.  

## AWS Bedrock 

Es un servicio que hace posible acceder por medio de un API modelos fundacionales de Amazon u otros proveedores como AI21 Labs, Anthropic entre otros. 

## Framework para la construcción de aplicaciones con LLMs LangChain 
Es un framework para el desarrollo de aplicaciones basadas en LLMs, el cual permite generar soluciones que tengan:  

1. Conciencia del contexto: permite conectar el modelo con fuentes de contexto o datos para una mejor interpretación de las solicitudes
   
3. Razonamiento: Con ayuda de los LLMs permite realizar razonamientos sobre como responder preguntas o que acciones se deben tomar teniendo como base el contexto que se provee

### En este laboratorio vamos a construir dos tipos de chatbot

1. Chatbot basico - sin contexto

2. Chatbot persona (Damos una identidad al chatbot)

### Chatbot basico - sin contexto 
En nuestro primer ejemplo vamos a utilizar los Chains de Langchain. Los Chains permiten conectar LLMs con otros componentes para crear aplicaciones que requieren flujos mas complejos. 

#### Instalación de dependencias
Lo primero sera instalar las dependencias necesarias para conectarnos con AWS y crear nuestra aplicación: 
1. Boto3: SDK de AWS para Python
2. Langchain: Framework para desarrollo de aplicaciones basadas en LLMs

In [1]:
!pip install boto3
!pip install langchain
# !pip install awscli //in case your are going to use google colab


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


#### Importar dependencias 

In [2]:
import json
import os
import sys
import boto3
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [3]:
# # en caso que vayas a utilizar Google Colab ejecuta esta celda 
# monta google drive 
from google.colab import drive
drive.mount('/content/drive')
# genera las credenciales 
!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/MyDrive/config/awscli.ini
path = "/content/drive/My Drive/config/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])

/content/drive/My Drive/config/awscli.ini


#### Inicializar nuestro LLM
- model_id: En este caso definimos el modelo que sera utilizado, recordemos que Bedrock nos permite utilizar modelos fundacionales de diversos proveedore

- credentials_profile_name: Debes añadir el perfil del AWS-CLI configurado en el notebook initialization

In [3]:
llm_claude = Bedrock(
  model_id="anthropic.claude-v2",
  model_kwargs={'max_tokens_to_sample': 200},
  credentials_profile_name="default", 
  region_name="us-east-1"
)

En este escenario veremos algunas utilidades de LangChain como:
- ConversationBufferWindowMemory facilita el almacenamiento de los mensajes intercambiados con el LLM]
- ConversationChain facilita el proceso de comunicación entre un usuario y el LLM

* El parametro k representa el tamaño de la ventana de contexto que vamos a tener para comunicarnos con el LLM, en este caso indicamos que se van a mantener en los prompt hasta 5 mensajes intercambiados en el pasado. 

In [4]:
memory = ConversationBufferWindowMemory( k=5, return_messages=True)
conversation = ConversationChain(
    llm=llm_claude,
    memory=memory
)

#### Primera prueba 
Vamos a realizar algunas preguntas a nuestro modelo sobre musica

In [5]:
conversation.run("En la escala de Do mayor cuales son las notas hasta Mi?")

' En la escala de Do mayor, las notas desde Do hasta Mi son:\n\nDo, Re, Mi\n\nLas notas de la escala de Do mayor son:\n\nDo, Re, Mi, Fa, Sol, La, Si, Do\n\nEmpezando en Do, las notas hasta llegar a Mi son:\n\nDo, Re, Mi'

In [6]:
conversation.run("Y despues de Mi?")

' Después de Mi, las siguientes notas en la escala de Do mayor son:\n\nFa, Sol, La, Si, Do\n\nAsí que la secuencia completa de notas desde Do hasta el siguiente Do es: \n\nDo, Re, Mi, Fa, Sol, La, Si, Do'

En este caso podemos revisar el historial de mensajes intercambiados

In [8]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='En la escala de Do mayor cuales son las notas hasta Mi?'),
  AIMessage(content=' En la escala de Do mayor, las notas desde Do hasta Mi son:\n\nDo, Re, Mi\n\nLas notas de la escala de Do mayor son:\n\nDo, Re, Mi, Fa, Sol, La, Si, Do\n\nEmpezando en Do, las notas hasta llegar a Mi son:\n\nDo, Re, Mi'),
  HumanMessage(content='Y despues de Mi?'),
  AIMessage(content=' Después de Mi, las siguientes notas en la escala de Do mayor son:\n\nFa, Sol, La, Si, Do\n\nAsí que la secuencia completa de notas desde Do hasta el siguiente Do es: \n\nDo, Re, Mi, Fa, Sol, La, Si, Do')]}

### Chatbot persona 
Ahora que hemos utilizado los chains vamos a proveer un poco de contexto para nuestro chatbot. 
En este caso vamos a interactuar con un experto en Café


In [35]:
claude_prompt = PromptTemplate.from_template("""
Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
""")

In [39]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("You are a coffee expert. Your goal is to give advice to people interested in establishing a coffee venture")
memory.chat_memory.add_ai_message("I am a coffee expert and I give advice about how to start coffee ventures")
conversation_persona = ConversationChain(
     llm=llm_claude, verbose=True, memory=memory
)
conversation_persona.prompt = claude_prompt

print(conversation.predict(input="What will be a nice name for a new coffee shop in Colombia?"))

 Here are some additional nice name ideas for a new coffee shop in Colombia:

- Café de la Sierra - "Café of the Mountains", referring to the mountainous terrain where coffee is grown.

- Café de la Cosecha - "Café of the Harvest", highlighting the fresh-picked coffee beans. 

- Café del Campo - "Café of the Countryside", evoking the rural coffee farms.

- Café Rosales - Using a common Colombian surname that sounds pleasant.

- Café del Sol - "Café of the Sun", connecting to Colombia's sunny weather.

- Café Colonial - Evoking Colombia's colonial architecture and history.

- Café del Parque - "Café of the Park", if it's located near a town square or park.

- Café Caribe - Referencing Colombia's Caribbean coastline. 

- Café Dulce - "Sweet Café", emphasizing quality and flavor.
